# 수행목표
- 사용자별 평균 정밀도 계산 코드를 개발한다.
# 수행단계
- 아래 데이터를 활용해서 사용자별 평균 정밀도 AP@K를 확인하는 코드를 개발한다.

|사용자|가위손|나 홀로 집에|대부|사운드 오브 뮤직|시네마 천국|아마데우스|죽은 시인의 사회|터미네이터 2:오리지널|
|---|---|---|---|---|---|---|---|---|
|1917|1|0|1|1|1|1|1|1|
|10418|1|1|1|1|0|0|1|1|
|1980|1|1|1|1|1|1|0|0|
|2277|1|1|0|1|1|0|1|1|
|1805|0|0|1|1|1|1|1|1|
|5136|1|0|1|1|0|1|1|1|
|1561|1|0|1|0|1|1|1|1|
|1105|1|1|1|0|1|0|1|1|
|1312|1|1|0|0|1|0|1|1|
|3189|1|0|1|0|1|1|0|1|

- 위 데이터는 KMRD-small 데이터에서 사용자별 평가를 10개만 한 사용자와 영화별 평점 개수가 [60, 200)인 영화를 간추린 내용 중 일부이다.
- 평균 정밀도 공식은 아래와 같다.
    - K는 추천한 영화 개수
    - m은 사용자가 좋아한(평점을 매긴) 개수
    - Precision@i 는 1개부터 K개까지 각각 정밀도
    - rel(i)는 i번째 영화를 좋아했는지(평점 있는지) 여부 (1 or 0)
$$ AP@K = \frac{1}{m} \sum^{K}_{i=1} Precision@i \cdot rel(i) $$
- 사용자 1917의 AP@4는 아래와 같다. (4개 영화는 가나다순 추천으로 (가위손, 나 홀로 집에, 대부, 사운드 오브 뮤직) 순으로 처리)

In [1]:
# Library

import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['font.family'] = 'AppleGothic'
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
data = {
    "사용자": [1917, 10418, 1980, 2277, 1805, 5136, 1561, 1105, 1312, 3189],
    "가위손": [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    "나 홀로 집에": [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    "대부": [1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    "사운드 오브 뮤직": [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    "시네마 천국": [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
    "아마데우스": [1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
    "죽은 시인의 사회": [1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
    "터미네이터 2:오리지널": [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
}

df = pd.DataFrame(data)
df

,사용자,가위손,나 홀로 집에,대부,사운드 오브 뮤직,시네마 천국,아마데우스,죽은 시인의 사회,터미네이터 2:오리지널
0,1917,1,0,1,1,1,1,1,1
1,10418,1,1,1,1,0,0,1,1
2,1980,1,1,1,1,1,1,0,0
3,2277,1,1,0,1,1,0,1,1
4,1805,0,0,1,1,1,1,1,1
5,5136,1,0,1,1,0,1,1,1
6,1561,1,0,1,0,1,1,1,1
7,1105,1,1,1,0,1,0,1,1
8,1312,1,1,0,0,1,0,1,1
9,3189,1,0,1,0,1,1,0,1


In [3]:
recommend_data = [
    ["가위손", "나 홀로 집에", "대부"],
    ["가위손", "나 홀로 집에", "대부", "사운드 오브 뮤직"],
    ["나 홀로 집에", "사운드 오브 뮤직", "아마데우스"],
    ["가위손", "대부", "터미네이터 2:오리지널"],
    ["가위손", "시네마 천국", "터미네이터 2:오리지널"],
    ["가위손", "죽은 시인의 사회", "터미네이터 2:오리지널"],
    ["아마데우스", "죽은 시인의 사회", "터미네이터 2:오리지널"],
]

In [4]:
# Precision@K

def precision_at_k(df, movies, k):
    precisions = {}
    for user in df["사용자"]:
        user_data = df[df["사용자"] == user]

        recommended_movies = movies
        relevant_movies = user_data.columns[user_data.iloc[0] == 1]

        precision = len(set(recommended_movies) & set(relevant_movies)) / len(recommended_movies)
        precisions[user] = precision

    return precisions

In [5]:
movies = ["가위손", "나 홀로 집에", "대부", "사운드 오브 뮤직"]

precision_results = precision_at_k(df, movies, k=len(movies))
precision_results

{1917: 0.75,
 10418: 1.0,
 1980: 1.0,
 2277: 0.75,
 1805: 0.5,
 5136: 0.75,
 1561: 0.5,
 1105: 0.75,
 1312: 0.5,
 3189: 0.5}

In [6]:
def ap_at_k(df, movies, k):
    aps = {}
    for user in df["사용자"]:
        user_data = df[df["사용자"] == user]

        recommended_movies = movies
        relevant_movies = user_data.columns[user_data.iloc[0] == 1]

        ap = 0.0
        num_hits = 0.0
        for i, movie in enumerate(recommended_movies, start=1):
            if movie in relevant_movies:
                num_hits += 1
                ap += num_hits / i
            if i >= k:
                break

        aps[user] = ap / len(relevant_movies) if len(relevant_movies) > 0 else 0.0

    return aps

In [7]:
result_ap = ap_at_k(df, movies, len(movies))
result_ap

{1917: 0.34523809523809523,
 10418: 0.6666666666666666,
 1980: 0.6666666666666666,
 2277: 0.4583333333333333,
 1805: 0.13888888888888887,
 5136: 0.40277777777777773,
 1561: 0.27777777777777773,
 1105: 0.5,
 1312: 0.4,
 3189: 0.3333333333333333}

In [8]:
map = np.mean(list(result_ap.values()))
map

np.float64(0.4189682539682539)

In [9]:
def map_at_k(df, movies):
    aps = ap_at_k(df, movies, len(movies))
    map_value = np.mean(list(aps.values()))

    return map_value

In [10]:
for movies in recommend_data:
    print(map_at_k(df, movies))

0.34158730158730155
0.4189682539682539
0.22111111111111112
0.41674603174603175
0.4211904761904762
0.4223015873015873
0.32452380952380955
